In [2]:
import h5py
import numpy as np
import shutil
from misc_utils.tensor_sampling_utils import sample_tensors

In [7]:

# predtrenovane vahy
weights_source_path = 'trained_weights/VGG_coco_SSD_512x512_iter_360000.h5'


# cesta pro nove vahy a zkopirovani
weights_destination_path = 'trained_weights/VGG_coco_SSD_512x512_iter_360000_coral2.h5'
shutil.copy(weights_source_path, weights_destination_path)

'trained_weights/VGG_coco_SSD_512x512_iter_360000_coral2.h5'

In [8]:
# nacteni puvodnich a zkpirovanych vah
weights_source_file = h5py.File(weights_source_path, 'r')
weights_destination_file = h5py.File(weights_destination_path)

In [9]:
# jmena vrstev
classifier_names = ['conv4_3_norm_mbox_conf',
                    'fc7_mbox_conf',
                    'conv6_2_mbox_conf',
                    'conv7_2_mbox_conf',
                    'conv8_2_mbox_conf',
                    'conv9_2_mbox_conf',
                    'conv10_2_mbox_conf']

In [10]:
# puvodni pocet trid + 1 
n_classes_source = 81

# novy pocet trid + 1
classes_of_interest = 13+1

for name in classifier_names:
    
    kernel = weights_source_file[name][name]['kernel:0'].value
    bias = weights_source_file[name][name]['bias:0'].value


    height, width, in_channels, out_channels = kernel.shape
    
    if isinstance(classes_of_interest, (list, tuple)):
        subsampling_indices = []
        for i in range(int(out_channels/n_classes_source)):
            indices = np.array(classes_of_interest) + i * n_classes_source
            subsampling_indices.append(indices)
        subsampling_indices = list(np.concatenate(subsampling_indices))
    elif isinstance(classes_of_interest, int):
        subsampling_indices = int(classes_of_interest * (out_channels/n_classes_source))
    else:
        raise ValueError("`classes_of_interest` must be either an integer or a list/tuple.")
    

    new_kernel, new_bias = sample_tensors(weights_list=[kernel, bias],
                                          sampling_instructions=[height, width, in_channels, subsampling_indices],
                                          axes=[[3]], 
                                          init=['gaussian', 'zeros'],
                                          mean=0.0,
                                          stddev=0.005)
    
    
    del weights_destination_file[name][name]['kernel:0']
    del weights_destination_file[name][name]['bias:0']

    weights_destination_file[name][name].create_dataset(name='kernel:0', data=new_kernel)
    weights_destination_file[name][name].create_dataset(name='bias:0', data=new_bias)

weights_destination_file.flush()